##### 1. 줄거리 키워드 kbert로 추출 -> 임베딩 후 TF-IDF 유사도
##### 2. 줄거리 키워드 Sbert로 추출 -> 임베딩 후 TF-IDF 유사도
##### 줄거리 키워드 + 장르를 합쳐 유사도 계산을 통한 추천

In [ ]:
!pip install konlpy

In [ ]:
!pip install nltk

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
import pandas as pd
import numpy as np
import os

from bs4 import BeautifulSoup
from urllib.request import urlopen

import re
from konlpy.tag import Okt, Kkma, Komoran
import networkx
from tqdm.autonotebook import tqdm

import seaborn as sns
import matplotlib.pyplot as plt

from gensim.models import word2vec

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_excel('/content/drive/MyDrive/캡스톤/뮤지컬/뮤지컬_TOP70.xlsx')
df = df.drop(df.columns[0], axis=1)
# df = df.drop(df.columns[2], axis=1)
df.head()

,Title,Text_clear_num,뮤지컬 이름,공연소개,줄거리,장르
0,"2023 〈스웨그에이지 : 외쳐, 조선〉 - 인천",인기 우리의 작은 외침이 세상을 바꾼파 우리의 작은 외침이 세상을 바꾼다 우리들의 ...,"2023 〈스웨그에이지 : 외쳐, 조선〉 - 인천",NaN,시조'가 국가이념인 상상 속의 '조선'\n삶의 고단함과 역경을 시조 속에 담아 훌훌...,"역사극, 판타지"
1,2023 이은결 더 일루션 마스터피스 - 이천,케 티토 으으으 빨 게숫으오거 림 보이저 니거 법 존 이천아트홀 주간 전기차 제자 ...,2023 이은결 더 일루션 마스터피스 - 이천,NaN,<THE ILLUSION>은 단순히 여러 다양한 마술들을 나열하며 보여주는 기존의 ...,"마술, 판타지"
2,2023 창작뮤지컬어워드 NEXT,창작뮤지컬어워드 충무마트센터가 가능성 있는 참작뮤지컬 발굴을 플랫품인 창작뮤지컬머워...,2023 창작뮤지컬어워드 NEXT,None,헌책방을 운영하는 에이미는 여느 날과 다름없이 책들에게 인사를 건네며 하루를 시작한...,드라마
3,2023 최현우 Answer - 대구,구수성아트피아 대극장 년간의 마술 노하우를 집대성한 완결판 친혀으 미술시로컬어온 여...,2023 최현우 Answer - 대구,NaN,"마술사로 걸어온 27년\n2,700여회 공연, 200만 명 관객 돌파\n세계가 인정...","마술, 판타지"
4,2023 최현우 Answer - 성남,적 서니 정남아트센터 오페라하우스 이하늘이엔티 빅탑엔터테인언트 지라온플레이 년간의 ...,2023 최현우 Answer - 성남,NaN,"마술사로 걸어온 27년\n2,700여회 공연, 200만 명 관객 돌파\n세계가 인정...","마술, 판타지"


In [ ]:
df1 = df[['Title', '줄거리', '장르']]
df1.head(3)

,Title,줄거리,장르
0,"2023 〈스웨그에이지 : 외쳐, 조선〉 - 인천",시조'가 국가이념인 상상 속의 '조선'\n삶의 고단함과 역경을 시조 속에 담아 훌훌...,"역사극, 판타지"
1,2023 이은결 더 일루션 마스터피스 - 이천,<THE ILLUSION>은 단순히 여러 다양한 마술들을 나열하며 보여주는 기존의 ...,"마술, 판타지"
2,2023 창작뮤지컬어워드 NEXT,헌책방을 운영하는 에이미는 여느 날과 다름없이 책들에게 인사를 건네며 하루를 시작한...,드라마


### 장르 + 키워드 유사도 (TF-IDF)

In [ ]:
!pip install keybert
!pip install kiwipiepy

In [ ]:
from keybert import KeyBERT
from kiwipiepy import Kiwi
from transformers import BertModel
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
model = BertModel.from_pretrained('skt/kobert-base-v1')
kw_model = KeyBERT(model)

##### keyparse_n_gram_range : 단어를 쪼개서 n_gram으로 추출
##### ex) (3,3)일 경우 3개의 단어 묶음인 단어구 추출
##### use_maxsum, use_mmr : 결과 다양성을 위한 파라미터

In [ ]:
# 영어 모두 지우기
def remove_english(text):
    text_without_english = re.sub(r'[a-zA-Z]', '', text)
    return text_without_english

stop_words = ['의', '가', '이', '은', '는', '들', '좀', '잘', '걍', '도', '를', '으로', '자', '에', '와', '하', '하다', '한', '여러분', '진짜']
def extract_keywords(text):
    keywords = kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 1), stop_words=stop_words, top_n=5, use_maxsum=True)
    return keywords

def noun_extractor(text):
    results = []
    result = kiwi.analyze(text)
    for token, pos, _, _ in result[0][0]:
       # 단어의 길이가 1보다 길고 명사만 추출
        if len(token) != 1 and (pos.startswith('N') or pos.startswith('SL')):
            results.append(token)
    return results

keyword_list = []
for index, row in df1.iterrows():
    text_without_english = remove_english(row['줄거리'])
    text = ' '.join(noun_extractor(text_without_english))
    keywords = extract_keywords(text)
    keyword_list.append(keywords)

In [ ]:
df1['키워드'] = keyword_list
df1['키워드']

<ipython-input-125-9511c21339ff>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['키워드'] = keyword_list


0     [(홍국, 0.3453), (시조대판서, 0.3636), (행복, 0.381), (...
1     [(다양, 0.4827), (형식, 0.5127), (유일, 0.5134), (의식...
2     [(대신, 0.4652), (그녀, 0.4994), (작가, 0.5007), (마이...
3     [(대한민국, 0.4212), (연인, 0.4248), (열광, 0.4252), (...
4     [(대한민국, 0.4212), (연인, 0.4248), (열광, 0.4252), (...
                            ...                        
64    [(속죄, 0.4101), (눈초리, 0.4265), (군영, 0.427), (아들...
65    [(꽃밭, 0.4473), (낙하산, 0.4639), (물속, 0.4799), (댄...
66    [(취미, 0.4126), (사진작가, 0.4185), (도시, 0.4327), (...
67    [(무대, 0.1948), (경험, 0.2145), (겨울왕국, 0.2663), (...
68    [(유럽, 0.3644), (전문직, 0.3939), (혁명, 0.4639), (아...
Name: 키워드, Length: 69, dtype: object

In [ ]:
df1['keywords'] = df1['키워드'].apply(lambda keywords: [keyword[0] for keyword in keywords])
# 공백으로 연결된 키워드로 변경
df1['keywords'] = df1['keywords'].apply(lambda keywords: ' '.join(keywords))

In [ ]:
df1.head(3)

,Title,줄거리,장르,키워드,keywords
0,"2023 〈스웨그에이지 : 외쳐, 조선〉 - 인천",시조'가 국가이념인 상상 속의 '조선'\n삶의 고단함과 역경을 시조 속에 담아 훌훌...,"역사극, 판타지","[(홍국, 0.3453), (시조대판서, 0.3636), (행복, 0.381), (...",홍국 시조대판서 행복 악행 조선
1,2023 이은결 더 일루션 마스터피스 - 이천,<THE ILLUSION>은 단순히 여러 다양한 마술들을 나열하며 보여주는 기존의 ...,"마술, 판타지","[(다양, 0.4827), (형식, 0.5127), (유일, 0.5134), (의식...",다양 형식 유일 의식 마술
2,2023 창작뮤지컬어워드 NEXT,헌책방을 운영하는 에이미는 여느 날과 다름없이 책들에게 인사를 건네며 하루를 시작한...,드라마,"[(대신, 0.4652), (그녀, 0.4994), (작가, 0.5007), (마이...",대신 그녀 작가 마이클 엠마


In [ ]:
# 줄거리에서 뽑힌 키워드 5개 + 장르 합쳐서 새로운 열 생성
df_data = df1.drop(df1[(df1['장르']=='') | (df1['keywords'] =='')].index)
df_data['genres_keywords'] = df1['장르'] + " " +(df1['keywords'])

In [ ]:
df_data['genres_keywords']

0        역사극, 판타지 홍국 시조대판서 행복 악행 조선
1            마술, 판타지 다양 형식 유일 의식 마술
2               드라마 대신 그녀 작가 마이클 엠마
3        마술, 판타지 대한민국 연인 열광 마술사 스케일
4        마술, 판타지 대한민국 연인 열광 마술사 스케일
                  ...              
64         역사극, 드라마 속죄 눈초리 군영 아들 조선
65          판타지, 액션 꽃밭 낙하산 물속 댄스 골드
66      드라마, 공감/힐링 취미 사진작가 도시 한국 오해
67             액션 무대 경험 겨울왕국 바다 제주도
68    드라마, 코믹, 감동 유럽 전문직 혁명 아내 자본주의
Name: genres_keywords, Length: 69, dtype: object

In [ ]:
tfidf_vector = TfidfVectorizer()

tfidf_matrix = tfidf_vector.fit_transform(df_data['genres_keywords']).toarray()
tfidf_matrix_feature = tfidf_vector.get_feature_names_out()
tfidf_matrix = pd.DataFrame(tfidf_matrix, columns=tfidf_matrix_feature, index=df1.Title)

In [ ]:
tfidf_matrix

In [ ]:
cossim = cosine_similarity(tfidf_matrix)
cossim_df = pd.DataFrame(cossim, index=df1.Title, columns=df1.Title)

# 유사도 높은 영화 반환
def top_k(target_title, matrix, items, k=10):
    rec_idx = matrix.loc[:, target_title].values.reshape(1, -1).argsort()[:, ::-1].flatten()[1:k+1]
    rec_title = items.iloc[rec_idx, :].Title.values
    rec_genre = items.iloc[rec_idx, :].장르.values
    rec_similarity = matrix.loc[target_title, rec_title].values
    return pd.DataFrame({'title': rec_title, 'genre': rec_genre, 'cosine_similarity': rec_similarity})

In [ ]:
top_k('뮤지컬 〈삼총사〉', cossim_df, df_data)

,title,genre,cosine_similarity
0,국립창극단 〈패왕별희〉,"로맨스, 역사극",0.205600
1,뮤지컬 〈레미제라블〉 - 부산,"역사극, 드라마, 감동",0.185761
2,뮤지컬 〈레미제라블〉,"역사극, 드라마, 감동",0.185761
3,뮤지컬 〈판〉,"코믹, 블랙코미디, 감동",0.166767
4,2023송승환의 오리지널 난타(대전),"코믹, 액션",0.149740
5,난타(NANTA) - 명동공연,"코믹, 액션",0.149740
6,난타(NANTA) - 홍대공연,"코믹, 액션",0.149740
7,뮤지컬 ＇벤허＇,"드라마, 역사극, 액션",0.130897
8,넌버벌 코미디 〈옹알스〉,"코믹, 액션",0.130079
9,쇼 뮤지컬 〈시스터즈 (SheStars!)〉,"코믹, 역사극, 감동",0.108135


In [ ]:
top_k('뮤지컬 〈쇼맨_어느 독재자의 네 번째 대역배우〉', cossim_df, df_data)

,title,genre,cosine_similarity
0,［뮤지컬 on stage］ 뮤지컬 〈쇼맨 : 어느 독재자의 네 번째 대역배우〉 - 화성,"드라마, 공감/힐링",1.000000
1,뮤지컬 〈빨래〉 - 대구,"드라마, 감동, 공감/힐링",0.192005
2,뮤지컬 〈빨래〉,"드라마, 감동, 공감/힐링",0.192005
3,뮤지컬 〈인사이드 윌리엄〉 2023,"드라마, 공감/힐링",0.186482
4,뮤지컬 〈칠칠〉,"드라마, 공감/힐링",0.182744
5,뮤지컬 〈문스토리〉,"드라마, 공감/힐링, 감동",0.173169
6,뮤지컬 〈렌트〉,"드라마, 로맨스, 공감/힐링",0.171491
7,라면에 파송송,"코미디, 공감/힐링",0.141320
8,뮤지컬 〈스모크〉,드라마,0.030572
9,뮤지컬 〈스토리오브마이라이프〉,"드라마, 감동",0.029810


### 장르 + 키워드 유사도 (Sbert)

In [ ]:
!pip install sentence_transformers

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('jhgan/ko-sroberta-multitask')

In [ ]:
import numpy as np
import itertools

from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer,
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

In [ ]:
df1.head()

,Title,줄거리,장르
0,"2023 〈스웨그에이지 : 외쳐, 조선〉 - 인천",시조'가 국가이념인 상상 속의 '조선'\n삶의 고단함과 역경을 시조 속에 담아 훌훌...,"역사극, 판타지"
1,2023 이은결 더 일루션 마스터피스 - 이천,<THE ILLUSION>은 단순히 여러 다양한 마술들을 나열하며 보여주는 기존의 ...,"마술, 판타지"
2,2023 창작뮤지컬어워드 NEXT,헌책방을 운영하는 에이미는 여느 날과 다름없이 책들에게 인사를 건네며 하루를 시작한...,드라마
3,2023 최현우 Answer - 대구,"마술사로 걸어온 27년\n2,700여회 공연, 200만 명 관객 돌파\n세계가 인정...","마술, 판타지"
4,2023 최현우 Answer - 성남,"마술사로 걸어온 27년\n2,700여회 공연, 200만 명 관객 돌파\n세계가 인정...","마술, 판타지"


In [ ]:
okt = Okt()

model = SentenceTransformer('jhgan/ko-sroberta-multitask')

def extract_keywords(text, model, n_gram_range=(1, 1), top_n=5):
    tokenized_doc = okt.pos(text)
    tokenized_nouns = ' '.join([word[0] for word in tokenized_doc if word[1] == 'Noun'])

    count = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_nouns])
    candidates = count.get_feature_names_out()

    doc_embedding = model.encode([text])
    candidate_embeddings = model.encode(candidates)

    distances = cosine_similarity(doc_embedding, candidate_embeddings)

    keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]
    return keywords

df1['keywords'] = df1['줄거리'].apply(lambda text: extract_keywords(text, model))


<ipython-input-67-008cc6305423>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['keywords'] = df1['줄거리'].apply(lambda text: extract_keywords(text, model))


In [ ]:
df1['keywords']

0         [홍국, 백성, 양반, 조선, 시조]
1         [표현, 형식, 공연, 매직, 마술]
2      [구절, 레도르, 작가, 에이미, 헌책방]
3      [최현우, 초대형, 최연, 마술, 마술사]
4      [최현우, 초대형, 최연, 마술, 마술사]
                ...           
64    [임진년, 순신, 수군, 거북선, 임진왜란]
65       [세계, 서커스, 웅장, 환상, 곡예]
66      [인생, 사진, 순간, 촬영, 사진작가]
67      [제주도, 상설, 무대, 공연, 뮤지컬]
68     [생산, 산업혁명, 생계, 산업화, 장인]
Name: keywords, Length: 69, dtype: object

In [ ]:
df1['keywords'] = df1['keywords'].apply(lambda keywords: ' '.join(keywords))

<ipython-input-69-4fce4414ea5f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['keywords'] = df1['keywords'].apply(lambda keywords: ' '.join(keywords))


In [ ]:
df1.head(3)

,Title,줄거리,장르,keywords
0,"2023 〈스웨그에이지 : 외쳐, 조선〉 - 인천",시조'가 국가이념인 상상 속의 '조선'\n삶의 고단함과 역경을 시조 속에 담아 훌훌...,"역사극, 판타지",홍국 백성 양반 조선 시조
1,2023 이은결 더 일루션 마스터피스 - 이천,<THE ILLUSION>은 단순히 여러 다양한 마술들을 나열하며 보여주는 기존의 ...,"마술, 판타지",표현 형식 공연 매직 마술
2,2023 창작뮤지컬어워드 NEXT,헌책방을 운영하는 에이미는 여느 날과 다름없이 책들에게 인사를 건네며 하루를 시작한...,드라마,구절 레도르 작가 에이미 헌책방


In [ ]:
df_data = df1.drop(df1[(df1['장르']=='') | (df1['keywords'] =='')].index)
df_data['genres_keywords'] = df1['장르'] + " " +(df1['keywords'])

In [ ]:
df_data['genres_keywords']

0           역사극, 판타지 홍국 백성 양반 조선 시조
1            마술, 판타지 표현 형식 공연 매직 마술
2             드라마 구절 레도르 작가 에이미 헌책방
3         마술, 판타지 최현우 초대형 최연 마술 마술사
4         마술, 판타지 최현우 초대형 최연 마술 마술사
                  ...              
64      역사극, 드라마 임진년 순신 수군 거북선 임진왜란
65          판타지, 액션 세계 서커스 웅장 환상 곡예
66      드라마, 공감/힐링 인생 사진 순간 촬영 사진작가
67              액션 제주도 상설 무대 공연 뮤지컬
68    드라마, 코믹, 감동 생산 산업혁명 생계 산업화 장인
Name: genres_keywords, Length: 69, dtype: object

In [ ]:
tfidf_vector = TfidfVectorizer()

tfidf_matrix = tfidf_vector.fit_transform(df_data['genres_keywords']).toarray()
tfidf_matrix_feature = tfidf_vector.get_feature_names_out()
tfidf_matrix = pd.DataFrame(tfidf_matrix, columns=tfidf_matrix_feature, index=df1.Title)

In [ ]:
cossim = cosine_similarity(tfidf_matrix)
cossim_df = pd.DataFrame(cossim, index=df1.Title, columns=df1.Title)

In [ ]:
# 유사도 높은 영화 반환
def top_k(target_title, matrix, items, k=5):
    rec_idx = matrix.loc[:, target_title].values.reshape(1, -1).argsort()[:, ::-1].flatten()[1:k+1]
    rec_title = items.iloc[rec_idx, :].Title.values
    rec_genre = items.iloc[rec_idx, :].장르.values
    rec_similarity = matrix.loc[target_title, rec_title].values
    return pd.DataFrame({'title': rec_title, 'genre': rec_genre, 'cosine_similarity': rec_similarity})

In [ ]:
top_k('뮤지컬 〈삼총사〉', cossim_df, df_data)

,title,genre,cosine_similarity
0,뮤지컬 〈마리 퀴리〉,"드라마, 역사극",0.369576
1,2023송승환의 오리지널 난타(대전),"코믹, 액션",0.148586
2,난타(NANTA) - 명동공연,"코믹, 액션",0.148586
3,난타(NANTA) - 홍대공연,"코믹, 액션",0.148586
4,넌버벌 코미디 〈옹알스〉,"코믹, 액션",0.129808


In [ ]:
top_k('뮤지컬 〈쇼맨_어느 독재자의 네 번째 대역배우〉', cossim_df, df_data)

,title,genre,cosine_similarity
0,［뮤지컬 on stage］ 뮤지컬 〈쇼맨 : 어느 독재자의 네 번째 대역배우〉 - 화성,"드라마, 공감/힐링",1.000000
1,뮤지컬 〈프리다〉,드라마,0.339087
2,뮤지컬 〈빨래〉 - 대구,"드라마, 감동, 공감/힐링",0.190931
3,뮤지컬 〈빨래〉,"드라마, 감동, 공감/힐링",0.190931
4,뮤지컬 〈렌트〉,"드라마, 로맨스, 공감/힐링",0.184234
